In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# ========== АНАЛИЗ ФАЙЛА И СРАВНЕНИЕ СЕНСОРОВ ==========

# Путь к общему финальному файлу
final_all_path = r'C:\Users\VKosinov\Desktop\GitHub\work\acceleration_error\final.csv'

print(f"{'='*80}")
print("АНАЛИЗ ФАЙЛА И СРАВНЕНИЕ СЕНСОРОВ")
print(f"{'='*80}")

try:
    # Загружаем данные из файла
    df = pd.read_csv(final_all_path, delimiter=';', encoding='cp1251')
    print(f"Файл загружен: {final_all_path}")
    print(f"Количество строк: {len(df)}")
    print(f"Количество столбцов: {len(df.columns)}")
    print(f"\nСтолбцы в файле:")
    for col in df.columns:
        print(f"  - {col}")
    
    # Проверяем наличие необходимых столбцов
    required_columns = ['sensor', 'axis', 'error_%', 'new_error_%', 'frequency', 'boost']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"\nВНИМАНИЕ: Отсутствуют необходимые столбцы: {missing_columns}")
        print("Анализ может быть неполным.")
    
    # Определяем уникальные сенсоры
    sensors = sorted(df['sensor'].unique())
    print(f"\nУникальные сенсоры: {sensors}")
    
    # Определяем уникальные оси
    axes = sorted(df['axis'].unique())
    print(f"Уникальные оси: {axes}")
    
    # Проверяем наличие данных для двух сенсоров
    if len(sensors) < 2:
        print("\nВНИМАНИЕ: В файле меньше 2 сенсоров. Сравнение невозможно.")
    else:
        print(f"\nСравниваем сенсоры: {sensors[0]} и {sensors[1]}")
        
        # Функция для преобразования чисел с запятой в float
        def convert_to_float(value):
            if isinstance(value, str):
                return float(value.replace(',', '.'))
            return float(value)
        
        # Преобразуем числовые столбцы
        numeric_columns = ['error_%', 'new_error_%', 'frequency', 'boost']
        for col in numeric_columns:
            if col in df.columns:
                try:
                    df[f'{col}_num'] = df[col].apply(convert_to_float)
                except Exception as e:
                    print(f"Ошибка при преобразовании столбца {col}: {e}")
                    df[f'{col}_num'] = np.nan
        
        # ========== ОБЩИЙ АНАЛИЗ ПО СЕНСОРАМ ==========
        print(f"\n{'='*80}")
        print("ОБЩАЯ СТАТИСТИКА ПО СЕНСОРАМ")
        print(f"{'='*80}")
        
        # Создаем отдельные DataFrame для каждого сенсора
        sensor_dfs = {}
        stats_summary = {}
        
        for sensor in sensors:
            sensor_df = df[df['sensor'] == sensor].copy()
            sensor_dfs[sensor] = sensor_df
            
            # Статистика для старой ошибки
            old_errors = sensor_df['error_%_num'].dropna()
            new_errors = sensor_df['new_error_%_num'].dropna()
            
            stats = {
                'old_max': old_errors.max() if len(old_errors) > 0 else np.nan,
                'old_mean': old_errors.mean() if len(old_errors) > 0 else np.nan,
                'old_median': old_errors.median() if len(old_errors) > 0 else np.nan,
                'old_below_1.5': (old_errors <= 1.5).sum() if len(old_errors) > 0 else 0,
                'old_total': len(old_errors),
                'new_max': new_errors.max() if len(new_errors) > 0 else np.nan,
                'new_mean': new_errors.mean() if len(new_errors) > 0 else np.nan,
                'new_median': new_errors.median() if len(new_errors) > 0 else np.nan,
                'new_below_1.5': (new_errors <= 1.5).sum() if len(new_errors) > 0 else 0,
                'new_total': len(new_errors),
            }
            
            stats_summary[sensor] = stats
            
            print(f"\nСенсор {sensor}:")
            print(f"  Количество записей: {len(sensor_df)}")
            print(f"  Оси: {sorted(sensor_df['axis'].unique())}")
            
            if len(old_errors) > 0:
                print(f"\n  Старые ошибки:")
                print(f"    Максимальная: {stats['old_max']:.4f}%")
                print(f"    Средняя: {stats['old_mean']:.4f}%")
                print(f"    Медиана: {stats['old_median']:.4f}%")
                print(f"    ≤1.5%: {stats['old_below_1.5']}/{stats['old_total']} ({stats['old_below_1.5']/stats['old_total']*100:.1f}%)")
            
            if len(new_errors) > 0:
                print(f"\n  Новые ошибки:")
                print(f"    Максимальная: {stats['new_max']:.4f}%")
                print(f"    Средняя: {stats['new_mean']:.4f}%")
                print(f"    Медиана: {stats['new_median']:.4f}%")
                print(f"    ≤1.5%: {stats['new_below_1.5']}/{stats['new_total']} ({stats['new_below_1.5']/stats['new_total']*100:.1f}%)")
            
            if len(old_errors) > 0 and len(new_errors) > 0:
                improvement_max = stats['old_max'] - stats['new_max']
                improvement_mean = stats['old_mean'] - stats['new_mean']
                improvement_count = stats['new_below_1.5'] - stats['old_below_1.5']
                
                print(f"\n  Улучшение:")
                print(f"    Макс. ошибка: {improvement_max:.4f}%")
                print(f"    Сред. ошибка: {improvement_mean:.4f}%")
                print(f"    Строк ≤1.5%: +{improvement_count}")
        
        # ========== ГРАФИКИ СРАВНЕНИЯ СЕНСОРОВ ==========
        
        # 1. Сравнение распределений ошибок до и после для обоих сенсоров
        fig1 = plt.figure(figsize=(16, 10))
        
        # Создаем цветовую схему для сенсоров
        colors = {'1': 'blue', '2': 'red'}
        sensor_labels = {sensor: f"Сенсор {sensor}" for sensor in sensors}
        
        # 1.1 Гистограммы старых ошибок для обоих сенсоров
        # Определяем цветовую схему
        # Для старых ошибок используем более темные/насыщенные цвета
        old_colors = {
            'sensor1': '#1f77b4',  # темно-синий для старого sensor1
            'sensor2': '#ff7f0e'   # темно-оранжевый для старого sensor2
        }

        # Для новых ошибок используем более светлые/ненасыщенные цвета
        new_colors = {
            'sensor1': '#aec7e8',  # светлый синий для нового sensor1
            'sensor2': '#ffbb78'   # светлый оранжевый для нового sensor2
        }

        # 1.1 Гистограммы старых ошибок для обоих сенсоров
        ax1 = plt.subplot(2, 3, 1)
        for sensor in sensors:
            if sensor in sensor_dfs:
                old_errors = sensor_dfs[sensor]['error_%_num'].dropna()
                if len(old_errors) > 0:
                    ax1.hist(old_errors, bins=30, alpha=0.7, color=old_colors.get(sensor, 'gray'),
                            label=f'{sensor_labels[sensor]} (макс: {stats_summary[sensor]["old_max"]:.2f}%)',
                            edgecolor='black', linewidth=0.5)

        ax1.axvline(x=1.5, color='red', linestyle='--', linewidth=2, label='Порог 1.5%')
        ax1.set_xlabel('Ошибка, %')
        ax1.set_ylabel('Количество наблюдений')
        ax1.set_title('Распределение старых ошибок (до корректировки)')
        ax1.legend(fontsize=9)
        ax1.grid(True, alpha=0.3)

        # 1.2 Гистограммы новых ошибок для обоих сенсоров
        ax2 = plt.subplot(2, 3, 2)
        for sensor in sensors:
            if sensor in sensor_dfs:
                new_errors = sensor_dfs[sensor]['new_error_%_num'].dropna()
                if len(new_errors) > 0:
                    ax2.hist(new_errors, bins=30, alpha=0.7, color=new_colors.get(sensor, 'gray'),
                            label=f'{sensor_labels[sensor]} (макс: {stats_summary[sensor]["new_max"]:.2f}%)',
                            edgecolor='black', linewidth=0.5)

        ax2.axvline(x=1.5, color='red', linestyle='--', linewidth=2, label='Порог 1.5%')
        ax2.set_xlabel('Ошибка, %')
        ax2.set_ylabel('Количество наблюдений')
        ax2.set_title('Распределение новых ошибок (после корректировки)')
        ax2.legend(fontsize=9)
        ax2.grid(True, alpha=0.3)
        
        # 1.3 Boxplot сравнения старых и новых ошибок по сенсорам
        ax3 = plt.subplot(2, 3, 3)
        boxplot_data = []
        boxplot_labels = []
        
        for sensor in sensors:
            if sensor in sensor_dfs:
                old_errors = sensor_dfs[sensor]['error_%_num'].dropna()
                new_errors = sensor_dfs[sensor]['new_error_%_num'].dropna()
                
                if len(old_errors) > 0:
                    boxplot_data.append(old_errors.values)
                    boxplot_labels.append(f'Сенсор {sensor}\nстарый')
                
                if len(new_errors) > 0:
                    boxplot_data.append(new_errors.values)
                    boxplot_labels.append(f'Сенсор {sensor}\nновый')
        
        bp = ax3.boxplot(boxplot_data, labels=boxplot_labels, patch_artist=True)
        
        # Раскрашиваем boxplot
        colors_box = []
        for label in boxplot_labels:
            if 'старый' in label:
                colors_box.append('lightblue' if '1' in label else 'lightcoral')
            else:
                colors_box.append('blue' if '1' in label else 'red')
        
        for patch, color in zip(bp['boxes'], colors_box):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        
        ax3.axhline(y=1.5, color='black', linestyle='--', linewidth=2, label='Порог 1.5%')
        ax3.set_ylabel('Ошибка, %')
        ax3.set_title('Boxplot сравнения ошибок по сенсорам')
        ax3.legend(fontsize=9)
        ax3.grid(True, alpha=0.3, axis='y')
        
        # 1.4 Кумулятивное распределение для сравнения
        ax4 = plt.subplot(2, 3, 4)
        for sensor in sensors:
            if sensor in sensor_dfs:
                # Старые ошибки
                old_errors = sensor_dfs[sensor]['error_%_num'].dropna()
                if len(old_errors) > 0:
                    sorted_old = np.sort(old_errors.values)
                    cumulative_old = np.arange(1, len(sorted_old) + 1) / len(sorted_old) * 100
                    ax4.plot(sorted_old, cumulative_old, color=colors.get(sensor, 'gray'), 
                             linestyle='--', linewidth=2, alpha=0.7, label=f'Сенсор {sensor} (старый)')
                
                # Новые ошибки
                new_errors = sensor_dfs[sensor]['new_error_%_num'].dropna()
                if len(new_errors) > 0:
                    sorted_new = np.sort(new_errors.values)
                    cumulative_new = np.arange(1, len(sorted_new) + 1) / len(sorted_new) * 100
                    ax4.plot(sorted_new, cumulative_new, color=colors.get(sensor, 'gray'), 
                             linewidth=2, alpha=0.7, label=f'Сенсор {sensor} (новый)')
        
        ax4.axvline(x=1.5, color='black', linestyle='--', linewidth=2)
        ax4.set_xlabel('Ошибка, %')
        ax4.set_ylabel('Процент строк с ошибкой ≤ X, %')
        ax4.set_title('Кумулятивное распределение ошибок')
        ax4.legend(fontsize=8, loc='lower right')
        ax4.grid(True, alpha=0.3)
        ax4.set_ylim([0, 105])
        
        # 1.5 Сравнение максимальных и средних ошибок
        ax5 = plt.subplot(2, 3, 5)
        
        metrics = ['Макс. ошибка', 'Сред. ошибка', 'Медиана ошибки']
        sensor_positions = np.arange(len(sensors))
        width = 0.35
        
        old_values_max = [stats_summary[sensor]['old_max'] for sensor in sensors]
        new_values_max = [stats_summary[sensor]['new_max'] for sensor in sensors]
        
        old_values_mean = [stats_summary[sensor]['old_mean'] for sensor in sensors]
        new_values_mean = [stats_summary[sensor]['new_mean'] for sensor in sensors]
        
        old_values_median = [stats_summary[sensor]['old_median'] for sensor in sensors]
        new_values_median = [stats_summary[sensor]['new_median'] for sensor in sensors]
        
        # Максимальные ошибки
        bars1 = ax5.bar(sensor_positions - width, old_values_max, width, 
                       label='Старые (макс)', color='lightblue', alpha=0.7)
        bars2 = ax5.bar(sensor_positions, new_values_max, width, 
                       label='Новые (макс)', color='blue', alpha=0.7)
        
        # Средние ошибки
        bars3 = ax5.bar(sensor_positions + width, old_values_mean, width, 
                       label='Старые (сред)', color='lightcoral', alpha=0.7)
        bars4 = ax5.bar(sensor_positions + 2*width, new_values_mean, width, 
                       label='Новые (сред)', color='red', alpha=0.7)
        
        ax5.axhline(y=1.5, color='black', linestyle='--', linewidth=2, label='Порог 1.5%')
        ax5.set_xlabel('Сенсор')
        ax5.set_ylabel('Ошибка, %')
        ax5.set_title('Сравнение статистик ошибок')
        ax5.set_xticks(sensor_positions + width/2)
        ax5.set_xticklabels([f'Сенсор {sensor}' for sensor in sensors])
        ax5.legend(fontsize=8)
        ax5.grid(True, alpha=0.3, axis='y')
        
        # 1.6 Процент строк с ошибкой ≤ 1.5%
        ax6 = plt.subplot(2, 3, 6)
        
        old_percent = []
        new_percent = []
        
        for sensor in sensors:
            stats = stats_summary[sensor]
            if stats['old_total'] > 0:
                old_percent.append(stats['old_below_1.5'] / stats['old_total'] * 100)
            else:
                old_percent.append(0)
            
            if stats['new_total'] > 0:
                new_percent.append(stats['new_below_1.5'] / stats['new_total'] * 100)
            else:
                new_percent.append(0)
        
        x_pos = np.arange(len(sensors))
        
        bars_old = ax6.bar(x_pos - width/2, old_percent, width, 
                          label='Старые ошибки', color='lightblue', alpha=0.7)
        bars_new = ax6.bar(x_pos + width/2, new_percent, width, 
                          label='Новые ошибки', color='blue', alpha=0.7)
        
        ax6.axhline(y=100, color='green', linestyle='--', linewidth=1, alpha=0.5, label='100%')
        ax6.set_xlabel('Сенсор')
        ax6.set_ylabel('Процент строк с error ≤ 1.5%')
        ax6.set_title('Эффективность корректировки')
        ax6.set_xticks(x_pos)
        ax6.set_xticklabels([f'Сенсор {sensor}' for sensor in sensors])
        ax6.set_ylim([0, 105])
        ax6.legend(fontsize=9)
        ax6.grid(True, alpha=0.3, axis='y')
        
        # Добавляем значения на столбцы
        for bar, value in zip(bars_old, old_percent):
            height = bar.get_height()
            ax6.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=9)
        
        for bar, value in zip(bars_new, new_percent):
            height = bar.get_height()
            ax6.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=9)
        
        plt.suptitle('Сравнение сенсоров: ошибки до и после корректировки', fontsize=16, y=0.98)
        plt.tight_layout()
        plt.show()
        
        # ========== АНАЛИЗ ПО ОСЯМ ==========
        print(f"\n{'='*80}")
        print("АНАЛИЗ ПО ОСЯМ")
        print(f"{'='*80}")
        
        fig2 = plt.figure(figsize=(15, 12))
        
        for idx, axis in enumerate(axes, 1):
            ax = plt.subplot(len(axes), 1, idx)
            
            axis_data = df[df['axis'] == axis].copy()
            
            # Собираем данные для boxplot
            boxplot_data_axis = []
            boxplot_labels_axis = []
            
            for sensor in sensors:
                sensor_axis_data = axis_data[axis_data['sensor'] == sensor]
                
                old_errors = sensor_axis_data['error_%_num'].dropna()
                new_errors = sensor_axis_data['new_error_%_num'].dropna()
                
                if len(old_errors) > 0:
                    boxplot_data_axis.append(old_errors.values)
                    boxplot_labels_axis.append(f'Сенсор {sensor}\nстарый')
                
                if len(new_errors) > 0:
                    boxplot_data_axis.append(new_errors.values)
                    boxplot_labels_axis.append(f'Сенсор {sensor}\nновый')
            
            if boxplot_data_axis:
                bp_axis = ax.boxplot(boxplot_data_axis, labels=boxplot_labels_axis, patch_artist=True)
                
                # Раскрашиваем boxplot для оси
                colors_axis = []
                for label in boxplot_labels_axis:
                    if 'старый' in label:
                        colors_axis.append('lightblue' if '1' in label else 'lightcoral')
                    else:
                        colors_axis.append('blue' if '1' in label else 'red')
                
                for patch, color in zip(bp_axis['boxes'], colors_axis):
                    patch.set_facecolor(color)
                    patch.set_alpha(0.7)
                
                ax.axhline(y=1.5, color='black', linestyle='--', linewidth=2, label='Порог 1.5%')
                ax.set_ylabel('Ошибка, %')
                ax.set_title(f'Ось {axis}: Сравнение ошибок по сенсорам')
                ax.legend(fontsize=9)
                ax.grid(True, alpha=0.3, axis='y')
                
                # Вывод статистики для оси
                print(f"\nОсь {axis}:")
                for sensor in sensors:
                    sensor_axis_data = axis_data[axis_data['sensor'] == sensor]
                    old_count = sensor_axis_data['error_%_num'].notna().sum()
                    new_count = sensor_axis_data['new_error_%_num'].notna().sum()
                    print(f"  Сенсор {sensor}: {old_count} старых записей, {new_count} новых записей")
            else:
                ax.text(0.5, 0.5, f'Нет данных для оси {axis}', 
                       ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'Ось {axis}: Нет данных')
        
        plt.suptitle('Сравнение ошибок по осям', fontsize=16, y=0.98)
        plt.tight_layout()
        plt.show()
        
        # ========== СРАВНЕНИЕ ЗАВИСИМОСТИ ОТ ЧАСТОТЫ ==========
        print(f"\n{'='*80}")
        print("АНАЛИЗ ЗАВИСИМОСТИ ОТ ЧАСТОТЫ")
        print(f"{'='*80}")
        
        fig3 = plt.figure(figsize=(15, 10))
        
        # 3.1 Зависимость старых ошибок от частоты
        ax1 = plt.subplot(2, 2, 1)
        for sensor in sensors:
            sensor_data = df[df['sensor'] == sensor].copy()
            if len(sensor_data) > 0:
                # Группируем по частоте
                freq_groups = sensor_data.groupby('frequency_num')
                freq_values = []
                error_means = []
                
                for freq, group in freq_groups:
                    errors = group['error_%_num'].dropna()
                    if len(errors) > 0:
                        freq_values.append(freq)
                        error_means.append(errors.mean())
                
                if freq_values and error_means:
                    ax1.scatter(freq_values, error_means, s=80, alpha=0.7, 
                               color=colors.get(sensor, 'gray'), label=f'Сенсор {sensor}')
                    # Линия тренда
                    if len(freq_values) > 1:
                        z = np.polyfit(freq_values, error_means, 1)
                        p = np.poly1d(z)
                        ax1.plot(freq_values, p(freq_values), color=colors.get(sensor, 'gray'), 
                                linestyle='--', alpha=0.5)
        
        ax1.axhline(y=1.5, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Порог 1.5%')
        ax1.set_xlabel('Частота')
        ax1.set_ylabel('Средняя ошибка, %')
        ax1.set_title('Зависимость старых ошибок от частоты')
        ax1.legend(fontsize=9)
        ax1.grid(True, alpha=0.3)
        
        # 3.2 Зависимость новых ошибок от частоты
        ax2 = plt.subplot(2, 2, 2)
        for sensor in sensors:
            sensor_data = df[df['sensor'] == sensor].copy()
            if len(sensor_data) > 0:
                # Группируем по частоте
                freq_groups = sensor_data.groupby('frequency_num')
                freq_values = []
                error_means = []
                
                for freq, group in freq_groups:
                    errors = group['new_error_%_num'].dropna()
                    if len(errors) > 0:
                        freq_values.append(freq)
                        error_means.append(errors.mean())
                
                if freq_values and error_means:
                    ax2.scatter(freq_values, error_means, s=80, alpha=0.7, 
                               color=colors.get(sensor, 'gray'), label=f'Сенсор {sensor}')
                    # Линия тренда
                    if len(freq_values) > 1:
                        z = np.polyfit(freq_values, error_means, 1)
                        p = np.poly1d(z)
                        ax2.plot(freq_values, p(freq_values), color=colors.get(sensor, 'gray'), 
                                linestyle='--', alpha=0.5)
        
        ax2.axhline(y=1.5, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Порог 1.5%')
        ax2.set_xlabel('Частота')
        ax2.set_ylabel('Средняя ошибка, %')
        ax2.set_title('Зависимость новых ошибок от частоты')
        ax2.legend(fontsize=9)
        ax2.grid(True, alpha=0.3)
        
        # 3.3 Сравнение улучшения по частотам
        ax3 = plt.subplot(2, 2, 3)
        improvement_data = {}
        
        for sensor in sensors:
            sensor_data = df[df['sensor'] == sensor].copy()
            if len(sensor_data) > 0:
                # Группируем по частоте
                freq_groups = sensor_data.groupby('frequency_num')
                freq_values = []
                improvements = []
                
                for freq, group in freq_groups:
                    old_errors = group['error_%_num'].dropna()
                    new_errors = group['new_error_%_num'].dropna()
                    
                    if len(old_errors) > 0 and len(new_errors) > 0:
                        old_mean = old_errors.mean()
                        new_mean = new_errors.mean()
                        improvement = old_mean - new_mean
                        
                        freq_values.append(freq)
                        improvements.append(improvement)
                
                if freq_values and improvements:
                    improvement_data[sensor] = (freq_values, improvements)
                    ax3.scatter(freq_values, improvements, s=80, alpha=0.7, 
                               color=colors.get(sensor, 'gray'), label=f'Сенсор {sensor}')
                    
                    # Линия тренда
                    if len(freq_values) > 1:
                        z = np.polyfit(freq_values, improvements, 1)
                        p = np.poly1d(z)
                        ax3.plot(freq_values, p(freq_values), color=colors.get(sensor, 'gray'), 
                                linestyle='--', alpha=0.5)
        
        ax3.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.3)
        ax3.set_xlabel('Частота')
        ax3.set_ylabel('Улучшение (старая - новая), %')
        ax3.set_title('Улучшение ошибок в зависимости от частоты')
        ax3.legend(fontsize=9)
        ax3.grid(True, alpha=0.3)
        
        # 3.4 Сводная таблица улучшений
        ax4 = plt.subplot(2, 2, 4)
        
        # Создаем сводную таблицу
        summary_text = "СВОДКА УЛУЧШЕНИЙ\n\n"
        
        for sensor in sensors:
            if sensor in stats_summary:
                stats = stats_summary[sensor]
                summary_text += f"Сенсор {sensor}:\n"
                summary_text += f"  Макс. ошибка: {stats['old_max']:.3f}% → {stats['new_max']:.3f}%"
                if not np.isnan(stats['old_max']) and not np.isnan(stats['new_max']):
                    improvement = stats['old_max'] - stats['new_max']
                    summary_text += f" ({improvement:+.3f}%)\n"
                else:
                    summary_text += "\n"
                
                summary_text += f"  Сред. ошибка: {stats['old_mean']:.3f}% → {stats['new_mean']:.3f}%"
                if not np.isnan(stats['old_mean']) and not np.isnan(stats['new_mean']):
                    improvement = stats['old_mean'] - stats['new_mean']
                    summary_text += f" ({improvement:+.3f}%)\n"
                else:
                    summary_text += "\n"
                
                if stats['old_total'] > 0 and stats['new_total'] > 0:
                    old_percent = stats['old_below_1.5'] / stats['old_total'] * 100
                    new_percent = stats['new_below_1.5'] / stats['new_total'] * 100
                    improvement_percent = new_percent - old_percent
                    
                    summary_text += f"  ≤1.5%: {old_percent:.1f}% → {new_percent:.1f}%"
                    summary_text += f" ({improvement_percent:+.1f}%)\n"
                summary_text += "\n"
        
        ax4.text(0.1, 0.95, summary_text, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        ax4.set_axis_off()
        ax4.set_title('Сводка улучшений по сенсорам')
        
        plt.suptitle('Анализ зависимости ошибок от частоты', fontsize=16, y=0.98)
        plt.tight_layout()
        plt.show()
        
        # ========== ВЫВОД ИТОГОВ ==========
        print(f"\n{'='*80}")
        print("ИТОГИ СРАВНЕНИЯ СЕНСОРОВ")
        print(f"{'='*80}")
        
        # Сравниваем сенсоры
        if len(sensors) >= 2:
            sensor1, sensor2 = sensors[0], sensors[1]
            
            if sensor1 in stats_summary and sensor2 in stats_summary:
                stats1 = stats_summary[sensor1]
                stats2 = stats_summary[sensor2]
                
                print(f"\nСравнение сенсора {sensor1} и сенсора {sensor2}:")
                
                # Сравнение старых ошибок
                print(f"\nСтарые ошибки (до корректировки):")
                print(f"  Макс. ошибка: {stats1['old_max']:.3f}% vs {stats2['old_max']:.3f}%")
                print(f"  Сред. ошибка: {stats1['old_mean']:.3f}% vs {stats2['old_mean']:.3f}%")
                
                if stats1['old_total'] > 0 and stats2['old_total'] > 0:
                    old_percent1 = stats1['old_below_1.5'] / stats1['old_total'] * 100
                    old_percent2 = stats2['old_below_1.5'] / stats2['old_total'] * 100
                    print(f"  ≤1.5%: {old_percent1:.1f}% vs {old_percent2:.1f}%")
                
                # Сравнение новых ошибок
                print(f"\nНовые ошибки (после корректировки):")
                print(f"  Макс. ошибка: {stats1['new_max']:.3f}% vs {stats2['new_max']:.3f}%")
                print(f"  Сред. ошибка: {stats1['new_mean']:.3f}% vs {stats2['new_mean']:.3f}%")
                
                if stats1['new_total'] > 0 and stats2['new_total'] > 0:
                    new_percent1 = stats1['new_below_1.5'] / stats1['new_total'] * 100
                    new_percent2 = stats2['new_below_1.5'] / stats2['new_total'] * 100
                    print(f"  ≤1.5%: {new_percent1:.1f}% vs {new_percent2:.1f}%")
                
                # Сравнение улучшений
                print(f"\nУлучшение после корректировки:")
                
                improvement_max1 = stats1['old_max'] - stats1['new_max']
                improvement_max2 = stats2['old_max'] - stats2['new_max']
                print(f"  Улучшение макс. ошибки: {improvement_max1:.3f}% vs {improvement_max2:.3f}%")
                
                improvement_mean1 = stats1['old_mean'] - stats1['new_mean']
                improvement_mean2 = stats2['old_mean'] - stats2['new_mean']
                print(f"  Улучшение сред. ошибки: {improvement_mean1:.3f}% vs {improvement_mean2:.3f}%")
                
                if stats1['old_total'] > 0 and stats2['old_total'] > 0:
                    improvement_percent1 = new_percent1 - old_percent1
                    improvement_percent2 = new_percent2 - old_percent2
                    print(f"  Улучшение % строк ≤1.5%: {improvement_percent1:+.1f}% vs {improvement_percent2:+.1f}%")
                
                # Определяем лучший сенсор
                print(f"\nОценка эффективности корректировки:")
                
                # По максимальному улучшению
                if improvement_max1 > improvement_max2:
                    print(f"  По максимальному улучшению лучше сенсор {sensor1}")
                elif improvement_max2 > improvement_max1:
                    print(f"  По максимальному улучшению лучше сенсор {sensor2}")
                else:
                    print(f"  По максимальному улучшению сенсоры равны")
                
                # По среднему улучшению
                if improvement_mean1 > improvement_mean2:
                    print(f"  По среднему улучшению лучше сенсор {sensor1}")
                elif improvement_mean2 > improvement_mean1:
                    print(f"  По среднему улучшению лучше сенсор {sensor2}")
                else:
                    print(f"  По среднему улучшению сенсоры равны")
                
                # По итоговой точности
                if stats1['new_max'] < stats2['new_max']:
                    print(f"  По итоговой максимальной ошибке лучше сенсор {sensor1}")
                elif stats2['new_max'] < stats1['new_max']:
                    print(f"  По итоговой максимальной ошибке лучше сенсор {sensor2}")
                else:
                    print(f"  По итоговой максимальной ошибке сенсоры равны")
        
        print(f"\n{'='*80}")
        print("АНАЛИЗ ЗАВЕРШЕН")
        print(f"{'='*80}")
        
except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден: {final_all_path}")
except Exception as e:
    print(f"ОШИБКА при анализе файла: {e}")
    import traceback
    traceback.print_exc()